# ChatGLM云端包
##简介
Colab选择chatglm-6b-int4模型。（无论是GPU还是CPU模式。）
阿里云选择chatglm-6b模型。（大模型速度快，int模型运行目前会报错。）
目前只有阿里云的显存支持训练，运行34后再运行2。

In [ ]:
#@title 0.查看显卡
#@markdown 查看显卡信息。（可跳过）
!nvidia-smi

In [ ]:
#@title 1.安装依赖
#@markdown 失败时请重试。（git库有时候链接不上很正常）
import os
!git config --global --unset http.proxy
!git config --global --unset https.proxy
!pip install --upgrade pip
#Colab地址
%cd /content
#阿里云地址
%cd /mnt/workspace
!git clone https://github.com/gluttony-10/ChatGLM-Lora.git
if os.path.exists('ChatGLM-Lora'):
 %cd ChatGLM-Lora
 !git pull
 !pip install -r requirements.txt
 #@markdown 选择模型
 MODEL = "chatglm-6b-int4" #@param ["chatglm-6b","chatglm-6b-int8","chatglm-6b-int4"]
 if MODEL == "chatglm-6b" :
  !git clone --progress -v https://huggingface.co/THUDM/chatglm-6b
 elif MODEL == "chatglm-6b-int8" :
  !git clone --progress -v https://huggingface.co/THUDM/chatglm-6b-int8
 elif MODEL == "chatglm-6b-int4" :
  !git clone --progress -v https://huggingface.co/THUDM/chatglm-6b-int4

In [ ]:
#@title 2.运行GLM
#@markdown 运行成功后，点击gradio网址打开GLM
if MODEL == "chatglm-6b" :
 !python webui.py --path chatglm-6b
elif MODEL == "chatglm-6b-int8" :
 !python webui.py --path chatglm-6b-int8
elif MODEL == "chatglm-6b-int4" :
 !python webui.py --path chatglm-6b-int4

In [ ]:
#@title 3.数据转换
#@markdown 9train.json是训练文本，可替换，注意格式
!python cover_alpaca2jsonl.py \
    --data_path 9train.json \
    --save_path 9train.jsonl
!python tokenize_dataset_rows.py \
    --jsonl_path 9train.jsonl \
    --save_path data/alpaca \
    --max_seq_length 200

In [ ]:
#@title 4.训练
#@markdown 参数可修改
!wandb off #阿里云用
!python finetune.py \
    --dataset_path data/alpaca \
    --lora_rank 32 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 1 \
    --max_steps 1000 \
    --save_steps 1000 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir output